In [1]:
import io
from os import listdir
from os.path import isfile, isdir, join
path = r"carpe_data"
if(isdir(path)):
    directorys = [d for d in listdir(path) if isdir(join(path, d))]
    files = [f for f in listdir(path) if isfile(join(path, f))]
else:
    raise None

In [3]:
import json
import os
import shutil

train_prefix = ["180116","171206","180117"]
test_prefix = ["180118","180310"]
val_prefix = ["180114"]
path_ = join(path, 'images')
files = [f for f in listdir(path_) if isfile(join(path_, f))]

In [ ]:
def f(prefix, files, d):
    if not isdir(join(path, d)):
        os.makedirs(join(path, d))
    if not isdir(join(path, f'{d}_segm_npz')):
        os.makedirs(join(path, f'{d}_segm_npz'))
    for file in files:
        if(file[:6] in prefix):
            shutil.copy(join(path, 'images', file) , join(path, d, file))
            shutil.copy(join(path, 'segm_npz', f'{file[:-3]}npz') , join(path, f'{d}_segm_npz', f'{file[:-3]}npz'))
    
f(train_prefix, files, 'train')
f(test_prefix, files, 'test')
f(val_prefix, files, 'val')

In [4]:
with open(join(path,'annotations','apollo_keypoints_24_train.json')) as setup_file:
        keypoints_24_train = json.load(setup_file)
with open(join(path,'annotations','apollo_keypoints_24_val.json')) as setup_file:
        keypoints_24_val = json.load(setup_file)
with open(join(path,'annotations','apollo_keypoints_66_train.json')) as setup_file:
        keypoints_66_train = json.load(setup_file)
with open(join(path,'annotations','apollo_keypoints_66_val.json')) as setup_file:
        keypoints_66_val = json.load(setup_file)

In [11]:
"""'images', 'annotations'"""
keypoints_24_train['images'][0]

{'coco_url': 'unknown',
 'file_name': '171206_034630104_Camera_5.jpg',
 'id': 34630104,
 'license': 1,
 'date_captured': 'unknown',
 'width': 3384,
 'height': 2710}

In [ ]:
from utils.openpifpaf_helper import CAR_SKELETON_24, CAR_SKELETON_66 

def add_skeleton(img, skeleton):
    num_links = 0
    links = [0] * len(skeleton) * 5
    for i, (u,v) in enumerate(skeleton):
        xu,yu,zu = img['keypoints'][(u-1)*3:(u-1)*3+3]
        xv,yv,zv = img['keypoints'][(v-1)*3:(v-1)*3+3]
        if(zu > 0 and zv > 0):
            num_links += 1
            links[i*5:i*5+5] = (xu,yu,xv,yv,zv)
        
    img['num_links'] = num_links
    img['links'] = links

In [ ]:
train_24 = {'info' : keypoints_24_train['info'], 'categories' : keypoints_24_train['categories']}
test_24  = {'info' : keypoints_24_train['info'], 'categories' : keypoints_24_train['categories']}
val_24   = {'info' : keypoints_24_train['info'], 'categories' : keypoints_24_train['categories']}
train_66 = {'info' : keypoints_66_train['info'], 'categories' : keypoints_66_train['categories']}
test_66  = {'info' : keypoints_66_train['info'], 'categories' : keypoints_66_train['categories']}
val_66   = {'info' : keypoints_66_train['info'], 'categories' : keypoints_66_train['categories']}

def f(prefix, j_24, j_66,  d):
    j_24['images'] = []
    j_66['images'] = []
    image_id_24 = []
    image_id_66  = []
    j_24['annotations'] = []
    j_66['annotations'] = []
    for img in keypoints_24_train['images']:
        if(str(img['file_name'])[:6] in prefix):
            j_24['images'].append(img)
            image_id_24.append(img['id'])
            
    for img in keypoints_24_val  ['images']:
        if(str(img['file_name'])[:6] in prefix):
            j_24['images'].append(img)
            image_id_24.append(img['id'])
            
    for img in keypoints_66_train['images']:
        if(str(img['file_name'])[:6] in prefix):
            j_66['images'].append(img)
            image_id_66.append(img['id'])
            
    for img in keypoints_66_val  ['images']:
        if(str(img['file_name'])[:6] in prefix):
            j_66['images'].append(img)
            image_id_66.append(img['id'])
    
    for img in keypoints_24_train['annotations']:
        if (img['iscrowd'] <= 0) and (img['image_id'] in image_id_24):
            add_skeleton(img, CAR_SKELETON_24)
            j_24['annotations'].append(img)
            
    for img in keypoints_24_val  ['annotations']:
        if (img['iscrowd'] <= 0) and (img['image_id'] in image_id_24):
            add_skeleton(img, CAR_SKELETON_24)
            j_24['annotations'].append(img)
            
    for img in keypoints_66_train['annotations']:
        if (img['iscrowd'] <= 0) and (img['image_id'] in image_id_66):
            add_skeleton(img, CAR_SKELETON_66)
            j_66['annotations'].append(img)
            
    for img in keypoints_66_val  ['annotations']:
        if (img['iscrowd'] <= 0) and (img['image_id'] in image_id_66):
            add_skeleton(img, CAR_SKELETON_66)
            j_66['annotations'].append(img)
            
            
    with open(os.path.join(path, r'annotations', f'keypoints_{d}_24.json'), "w") as outfile:
        outfile.write(json.dumps(j_24))
    with open(os.path.join(path, 'annotations', f'keypoints_{d}_66.json'), "w") as outfile:
        outfile.write(json.dumps(j_66))
    
    
f(train_prefix, train_24, train_66, 'train')
f(test_prefix, test_24, test_66, 'test')
f(val_prefix, val_24, val_66, 'val')